In [36]:
!pip install numpy
!pip install scipy
!pip install pandas
!pip install multiprocessing
!pip install statsmodels
!pip install matplotlib


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached multiprocessing-2.6.2.1.tar.gz (108 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [7 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/kk/1fzwcqps6dd_3dt2hsjct0tr_85vzj/T/pip-install-asuqeo71/multiprocessing_c11715a4abba4d598242d804d446e2a8/setup.py", line 94
          print 'Macros:'
                ^
      SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Macros:')?
      [end 

In [37]:
import random 
import numpy as np
import scipy as sp
import pandas as pd
from multiprocessing import Pool
from statsmodels.stats.multitest import multipletests as mult  #MULTIPLE TESTING MODULE 
from matplotlib import pyplot as plt

In [38]:
MaxReps=100
PREFIX="REACTOME_"#DB to explore for enrichment (see the DB file for other formats "GO_", others...)

In [39]:
#'''
file="/Users/dolanpt/GitHub_local/doggo/msigdb.v2022.1.Mm.symbols.gmt.txt"#CHANGE THIS TO YOUR LOCATION. RUN BLOCK ONCE (to make pickles) THEN COMMENT OUT. 
with open(file,'r') as inf:
    alllines=[line.strip().split("\t") for line in inf]
    DF=pd.concat([pd.DataFrame(zip([l[0]]*len(l[2:]),l[2:]),columns=["term","genes"]) for l in alllines])
pd.to_pickle(DF,"/Users/dolanpt/GitHub_local/doggo/msigdb.v2022.1.Mm.symbols.gmt.pkl")

In [40]:
MSigDB=pd.read_pickle("/Users/dolanpt/GitHub_local/doggo/msigdb.v2022.1.Mm.symbols.gmt.pkl")#CHANGE TO YOUR LOCALE (MATCH ABOVE)

In [41]:
#This inputData file should be a long table with genes as rows and the data (parameters for filtering and scoring (pval etc.)) for each as columns. 

inputData= "/Users/dolanpt/GitHub_local/doggo/Myeloid_cells_updated_0.4res.all_markers_genes.csv"
data=pd.read_table(inputData,delimiter=",")
data

,Unnamed: 0,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
0,Ms4a7,0.000000,2.294176,0.842,0.352,0.0,0,Ms4a7
1,C1qa,0.000000,1.706938,0.946,0.537,0.0,0,C1qa
2,C1qc,0.000000,1.533026,0.957,0.578,0.0,0,C1qc
3,C1qb,0.000000,1.633743,0.972,0.634,0.0,0,C1qb
4,Mrc1,0.000000,0.921698,0.770,0.439,0.0,0,Mrc1
...,...,...,...,...,...,...,...,...
56743,Gid8.6,0.009888,0.565666,0.299,0.216,1.0,7,Gid8
56744,Dhx8.5,0.009896,-0.666431,0.117,0.200,1.0,7,Dhx8
56745,Slc30a5.2,0.009907,-0.278236,0.104,0.184,1.0,7,Slc30a5
56746,Zbtb21.3,0.009914,-1.144930,0.065,0.135,1.0,7,Zbtb21


In [43]:
def computeEnrichment(t,d,P="b",ids="ext_gene",tailed=2):#
    threshold=21
    print(t)
    DBgenes=MSigDB.loc[MSigDB.term==t].genes
    #print(DBgenes)
    totalMean=d[P].values.mean()    #population mean
    #print(totalMean)
    targets=d.loc[d[ids].isin(DBgenes.values)]
    #print(targets)
    nrowS=targets.shape[0]          
    if nrowS>2:#If more that two genes in term set...
        #print(t)        
        #print(meanSelect)
        nrowT=d.shape[0]
        meanVar=totalMean-targets[P].abs().mean()       #mean deviation from mean value. 
        meanSelect=totalMean-targets[P].values.mean()   #difference between population mean and mean of sample
        positives=0
        denom=0
        if tailed==1:
            if meanSelect < 0:#choose which tail to test...
                while (positives<threshold)&(denom<MaxReps):
                    denom+=1
                    diffSample=totalMean-d[P].iloc[np.random.randint(0,nrowT,nrowS)].mean()
                    positives+=int(diffSample<meanSelect)
            else:
                while (positives<threshold)&(denom<MaxReps):
                    denom+=1                
                    diffSample=totalMean-d[P].iloc[np.random.randint(0,nrowT,nrowS)].mean()
                    positives+=int(diffSample>meanSelect)
        elif tailed==2:
            #print("tailed==2")
            while (positives<threshold)&(denom<MaxReps):
                denom+=1
                diffSample=totalMean-d[P].iloc[np.random.randint(0,nrowT,nrowS)].abs().mean()
                positives+=int(diffSample<meanVar)
        else:
            print("Must use 1 or 2 tail.")
        #print(positives/denom)
        #print((meanSelect-totalMean)/totalMean)
    else:
        #print("No hits. Skipping Term.")
        return(1.0,1.0,1.0,1.0)
    return([positives/denom, meanSelect, meanVar, nrowS])

In [44]:

#This is the main block, and needs to be edited based on your data and the parameter (score column you will be sampling in your data.)
def f(term,inputd):
    DF=pd.DataFrame([term,computeEnrichment(term,data,"avg_log2FC","gene")],index=["term","pBS"]).transpose()
    return(DF)

def performSampling(inputd,outputn):
#Run across all terms. 
    StatResults=pd.DataFrame()
    for term in MSigDB.term.unique():
        if term.startswith(PREFIX):
            result=f(term,inputd)
            StatResults=pd.concat([StatResults,pd.DataFrame([term,result.pBS[0][0],result.pBS[0][1],result.pBS[0][2],result.pBS[0][3]]).transpose()], ignore_index=True)
    StatResults.columns=["term","pBS","meanStat","meanVar","N"]            

    pcorr=mult(list(StatResults.pBS))
    StatResults['adjPbs']=pcorr[1]
    StatResults['sig']=pcorr[0]
    StatResults
    StatResults.to_csv(outputn)

for cluster in range(0,8):
    print(cluster)
    performSampling(data.loc[data.cluster==cluster],str(cluster)+"_Avglog2fc_pBS.csv")

0
REACTOME_INTERLEUKIN_6_SIGNALING
REACTOME_APOPTOSIS
REACTOME_HEMOSTASIS
REACTOME_INTRINSIC_PATHWAY_FOR_APOPTOSIS
REACTOME_MAPK3_ERK1_ACTIVATION
REACTOME_TRANSLESION_SYNTHESIS_BY_Y_FAMILY_DNA_POLYMERASES_BYPASSES_LESIONS_ON_DNA_TEMPLATE
REACTOME_RECOGNITION_OF_DNA_DAMAGE_BY_PCNA_CONTAINING_REPLICATION_COMPLEX
REACTOME_TRANSLESION_SYNTHESIS_BY_POLH
REACTOME_CLEAVAGE_OF_THE_DAMAGED_PYRIMIDINE
REACTOME_DISPLACEMENT_OF_DNA_GLYCOSYLASE_BY_APEX1
REACTOME_POLB_DEPENDENT_LONG_PATCH_BASE_EXCISION_REPAIR
REACTOME_RESOLUTION_OF_AP_SITES_VIA_THE_MULTIPLE_NUCLEOTIDE_PATCH_REPLACEMENT_PATHWAY
REACTOME_ACTIVATION_OF_BAD_AND_TRANSLOCATION_TO_MITOCHONDRIA
REACTOME_BH3_ONLY_PROTEINS_ASSOCIATE_WITH_AND_INACTIVATE_ANTI_APOPTOTIC_BCL_2_MEMBERS
REACTOME_RELEASE_OF_APOPTOTIC_FACTORS_FROM_THE_MITOCHONDRIA
REACTOME_FORMATION_OF_APOPTOSOME
REACTOME_ACTIVATION_OF_CASPASES_THROUGH_APOPTOSOME_MEDIATED_CLEAVAGE
REACTOME_CYTOCHROME_C_MEDIATED_APOPTOTIC_RESPONSE
REACTOME_APOPTOTIC_CLEAVAGE_OF_CELLULAR_PROTEINS
REACT

In [ ]:
pcorr=mult(list(StatResults.pBS))
StatResults['adjPbs']=pcorr[1]
StatResults['sig']=pcorr[0]
StatResults
StatResults.to_csv("WT_RD_37_pBS.csv")


/Users/dolanpt/Library/Python/3.9/lib/python/site-packages/statsmodels/stats/multitest.py:186: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


The code below still doesn't work... not sure why. But this would make it faster. Perhaps also vectorize could be useful? 

In [ ]:
def f(term):
    DF=pd.DataFrame([term,computeEnrichment(term,data,"b")],index=["term","pBS"]).transpose()
    return(DF)

In [ ]:
if __name__ == '__main__':#Wrapped this f() function in a parallelization to speed up. Can tune pool size for computer. 
    with Pool(5) as p:
        StatResults=pd.concat(p.map(f, [term for term in MSigDB.term.unique() if term.startswith(PREFIX)][1:100]))#Just running on the first 100 to test, runs forever, throws errors
    pcorr=mult(list(StatResults.pBS))
    StatResults['adjPbs']=pcorr[1]
    StatResults['sig']=pcorr[0]
    StatResults
    PVMax.to_csv("PVMax.csv")#name output file approriately

Process SpawnPoolWorker-58:
Process SpawnPoolWorker-57:
Process SpawnPoolWorker-60:
Process SpawnPoolWorker-56:
Process SpawnPoolWorker-59:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/dolanpt/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/dolanpt/anaconda3/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/dolanpt/anaconda3/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Users/dolanpt/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/dolanpt/anaconda3/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/d

KeyboardInterrupt: 